In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\Selva Karthik\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Small, fast BERT-based model

In [ ]:
def create_context_from_csv(df):
    row_texts = []
    for _, row in df.iterrows():
        sentence = sentence = f"Student_ID {row['Student_ID']}: {row['Name']} is {row['Age']} years old, enrolled in {row['Course']}, scored {row['Marks']} marks, and got grade {row['Grade']}."
        row_texts.append(sentence)
    return row_texts

def get_best_match_answer(question, row_texts):
    # Encode both question and all rows
    question_embedding = model.encode(question, convert_to_tensor=True)
    row_embeddings = model.encode(row_texts, convert_to_tensor=True)

    # Compute cosine similarity
    scores = util.pytorch_cos_sim(question_embedding, row_embeddings)[0]
    top_idx = scores.argmax().item()
    best_score = scores[top_idx].item()

    # Optional: filter out low-confidence matches
    if best_score < 0.4:
        return "🤖 Sorry, I couldn not find a confident match. Please rephrase your question."

    return row_texts[top_idx]

In [ ]:
print("BERT-Powered Semantic Student Chatbot")
csv_path = r"/content/DL.xlsx"
df = pd.read_excel(csv_path)
row_texts = create_context_from_csv(df)
while True:
    question = input(" Your Question: ")
    if question.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break
    answer = get_best_match_answer(question, row_texts)
    print(f"🤖 Answer: {answer}\n")